In [1]:
from working import *

In [69]:
from abc import ABC as _ABC, ABCMeta as _ABCMeta
import inspect as _inspect
# import weakref as _weakref

# from everest.cascade import (
#     Cascade as _Cascade,
#     Inputs as _Inputs,
#     )
from everest import (
    reseed as _reseed,
    wordhash as _wordhash,
    )
# from everest.h5anchor import disk as _disk

class Schema(_ABCMeta):
    userdefined = False
    schemas = dict()
    inputs = None
    hashID = None
    def __new__(meta, name, bases, dic):
        schema = super().__new__(meta, name, bases, dic)
        if schema.userdefined:
#             schema.script = script = _disk.ToOpen(inspect.getfile(schema))()
            schema.script = script = str(_reseed.digits(12))
            hashID = schema.hashID = _wordhash.get_random_proper(2, seed = script)
        else:
            hashID = schema.hashID = schema.__name__
        if not hashID in (pre := meta.schemas):
            pre[hashID] = schema
#             schema.inputs = _Inputs(schema.__init__, name = schema.__name__)
            schema.sig = _inspect.signature(schema)
            schema.hashID = hashID
        return pre[hashID]
    def __init__(schema, *args, **kwargs):
        schema.cases = dict()
    def _case_new__(schema, *args, **kwargs):
        inputs = schema.inputs.copy(*args, **kwargs)
        inhash = inputs.hashID
        if inhash in (cases := schema.cases):
            case = cases[inhash]
        else:
            case = cases[inhash] = super().__new__(
                type(schema),
                f"{schema.__name__}:{inhash}",
                (schema,),
                {**schema.__dict__, **dict(
                    inputs = inputs,
                    hashID = schema.hashID + ':' + inhash,
                    )},
                )
        return case
    def __getitem__(schema, inputs):
        return schema._case_new__(**inputs)
    def __call__(schema, *args, **kwargs):
        if type(schema) is Schema:
            case = schema._case_new__(*args, **kwargs)

class Case(Schema):
    def __call__(case, *args, **kwargs):
        if hasattr(case, 'process_inputs'):
            args, kwargs = case.process_inputs(args, kwargs)
        inputs = case.inputs.copy(*args, **kwargs)
        inst = super().__call__(*inputs.args, **inputs.kwargs)
        inst.hashID = case.hashID + ';' + str(_reseed.digits(12))
        return inst

class MySchema(_ABC, metaclass = Schema):
    def __init__(self,
            foo = 1,
            bah = 2,
            qux = 3,
            ):
        print(foo, bah, qux)

# class Funcy(_ABC, metaclass = Schema):
#     def __init__(self,
#             *args,
#             **kwargs
#             ):
        

# class Ur(Funcy):
#     ...

# class Base(Funcy):
#     ...

# class Derived(Funcy):
#     ...

making new


In [70]:
mycase = MySchema[{}]

In [64]:
mycase(foo = 10)

10 2 3


<__main__.MySchema:inelegance-autocracies at 0x1ea8abcc970>

In [68]:
mycase = MySchema[dict(foo = 100)]

In [66]:
mycase.hashID

'MySchema:incommunicado-calcification'

In [71]:
_Inputs

everest.cascade.inputs.Inputs

In [74]:
def myfn(
        a,
        _b,
        ):
    ...

In [75]:
_Inputs(myfn)

Inputs{None}(
    a = ReqArg(a: <class 'object'>)
    )

In [29]:
class MyMeta(type):
    def __new__(cls, *args):
        print(args)
        return super().__new__(cls, *args)
class MyClass(metaclass = MyMeta):
    print('foo')

foo
('MyClass', (), {'__module__': '__main__', '__qualname__': 'MyClass'})


In [13]:
mycase = Case(MySchema, foo = 10)[0]

making new


In [17]:
mycase

__main__.MySchema:disagreement-substitutes

In [14]:
mycase = Case(MySchema, foo = 10)[0]

TypeError: type.__init__() takes no keyword arguments

In [ ]:
class MySchema2(_ABC, metaclass = Case):
    def __init__(self,
            foo,
            bah,
            qux = 5,
            ):
        ...

In [ ]:
type(MySchema2)

In [ ]:
class MyClass:
    foo : int = 1

In [ ]:
MyClass.__annotations__

In [ ]:
dir(MyClass)

In [ ]:
weakref.WeakValueDictionary()

In [ ]:
myweak = weakref.WeakValueDictionary()

In [ ]:
myweak.setdefault()

In [ ]:
dict.setdefault

In [ ]:
Funcy(a = 1)

In [ ]:
class A:
    ...
class B:
    ...
class C(A, B):
    ...

In [ ]:
import weakref

In [ ]:
ref = weakref.ref(C)

In [ ]:
ref()

In [ ]:
del C

In [ ]:
class C(A, B):
    ...

In [ ]:
ref()

In [ ]:
C